In [17]:
from ultralytics import YOLO
import cv2
import numpy as np

In [18]:
def find_contours_from_yolo(img_path, model_path):
    # Load the YOLOv8 model
    model = YOLO(model_path)

    # Load and preprocess the input image
    img = cv2.imread(img_path)
    print(img.shape)

    # Check if the image is loaded correctly
    if img is None:
        raise ValueError("Image not found or the path is incorrect.")
    
    print(f"Image shape: {img.shape}")

    # Check if the image is grayscale or color
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        
    H, W, _ = img.shape

    # Inference with the YOLOv8 model
    results = model(img)

    contours_list = []
    bounding_rects = []

    # Process each detection in the results
    for result in results:
        for j, mask in enumerate(result.masks.data):
            mask = mask.cpu().numpy().astype(np.uint8) * 255  # Convert to 8-bit binary mask

            # Check if mask is empty
            if mask.size == 0:
                raise ValueError("Mask is empty. Please check the model's output.")
            
            print(f"Mask shape before resize: {mask.shape}")

            # Resize mask to the original image size
            mask = cv2.resize(mask, (W, H))

            print(f"Mask shape after resize: {mask.shape}")

            # Find the contours of the mask
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if not contours:
                raise ValueError("No contours found in the mask. Please check the mask and try again.")
            
            contours_list.append(contours)
            
            # Compute bounding rectangles for each contour
            for contour in contours:
                x, y, width, height = cv2.boundingRect(contour)
                
                if width < 50 or height < 50:  # Adjust the threshold values as needed
                    print(f"Small bounding box ignored: ({x}, {y}, {width}, {height})")
                    continue
                bounding_rects.append((x, y, width, height))
                print(bounding_rects)
                
                # Optional: Draw the contour and bounding rectangle on the original image for visualization
                cv2.drawContours(img, [contour], -1, (0, 255, 0), 10)
                cv2.rectangle(img, (x, y), (x+width, y+height), (255, 0, 0), 10)
                
    max_width = 800
    max_height = 600
    img_resized = resize_image_to_fit_window(img, max_width, max_height)

    cv2.imshow('Mask Contours and Bounding Rectangles', img_resized)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
            
    return contours_list, bounding_rects, img_resized

In [19]:
def resize_image_to_fit_window(image, max_width, max_height):
    height, width = image.shape[:2]
    if width > max_width or height > max_height:
        scaling_factor = min(max_width / width, max_height / height)
        new_size = (int(width * scaling_factor), int(height * scaling_factor))
        return cv2.resize(image, new_size)
    else:
        return image

In [20]:
def cropping(bounding_rect, img_path):
    # for i, rect in enumerate(bounding_rects):
    x, y, w, h = bounding_rects[0]
    x1 = x+int(w/2)
    print(x1)
    y1 = y+int(h/2)
    print(y1)
    add_w = int(2051/2)
    print(add_w)
    add_h = int(2811/2)
    print(add_h)
    x2 = x1-add_w
    print(x2)
    y2 = y1-add_h
    print(y2)
    h1 = 2811
    print(h1)
    w1 = 2051
    print(w1)

    image = cv2.imread(img_path)
    # sub_rect = image[y2:y2+h, x2:x2+w]
    # resized_sub_rect = cv2.resize(sub_rect, (w1, h1))
    # cv2.rectangle(image, (x2, y2), (x2+w1, y2+h1), (255, 0, 0), 10)
    # x2 = max(0, x2)
    # y2 = max(0, y2)
    # end_x = min(image.shape[1], x2 + w1)
    # end_y = min(image.shape[0], y2 + h1)
    
    image = image[y2:y2+h1, x2:x2+w1]
    view_image("Resized",image)
    return image

In [21]:
def view_image(window_name, image, max_width=400, max_height=200):
    # Get the dimensions of the image
    image = resize_image_to_fit_window(image, max_width, max_height)
    
    # Display the image
    cv2.imshow(window_name, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [27]:
img_path = input("Enter image path: ")
model_path = '/Users/KIIT/Desktop/Research/CTR&TI/Models/runs/segment/train32/weights/last.pt'  # Path to your trained YOLOv8 model


contours_list, bounding_rects, img= find_contours_from_yolo(img_path, model_path)
cropped_image = cropping(bounding_rects,img_path)
print(cropped_image)
view_image("Original",img)
view_image("Resized",cropped_image)

# cv2.imwrite('original_sub_rectangle.jpg', sub_rect)
cv2.imwrite('resized.jpg', cropped_image)
# # for i, contours in enumerate(contours_list):
# #     print(f"Contours for mask {i}: {contours}")

# length, breadth, volume, shell_weight, grade = predict_cocoon_1(bounding_rects)
# print(f"Length: {length} cm, Breadth: {breadth} cm, Volume: {volume} cm^3, Shell Weight: {shell_weight} g, Grade: {grade}")
# # csv.data[length, breadth, volume, shell_weight, grade]

Enter image path:  C:/Users/KIIT/Desktop/Research/CTR&TI/Models/dataset/25Cocoons/DSC03610.JPG


(3672, 4896, 3)
Image shape: (3672, 4896, 3)

0: 480x640 1 Cocoon, 240.2ms
Speed: 8.5ms preprocess, 240.2ms inference, 7.0ms postprocess per image at shape (1, 3, 480, 640)
Mask shape before resize: (480, 640)
Mask shape after resize: (3672, 4896)
[(2092, 1297, 666, 1124)]
2425
1859
1025
1405
1400
454
2811
2051
[[[ 85  85  85]
  [ 90  90  90]
  [ 91  91  91]
  ...
  [104 102 101]
  [102 100  99]
  [ 99  97  96]]

 [[ 87  87  87]
  [ 92  92  92]
  [100 100 100]
  ...
  [ 99  97  96]
  [ 92  90  89]
  [ 96  94  93]]

 [[ 88  91  89]
  [ 94  97  95]
  [107 108 106]
  ...
  [ 91  92  90]
  [ 89  90  88]
  [ 87  88  86]]

 ...

 [[ 49  55  54]
  [ 49  55  54]
  [ 49  55  54]
  ...
  [ 54  63  60]
  [ 51  60  57]
  [ 49  58  55]]

 [[ 46  52  51]
  [ 46  52  51]
  [ 44  50  49]
  ...
  [ 49  60  57]
  [ 48  59  56]
  [ 49  58  55]]

 [[ 47  52  51]
  [ 46  51  50]
  [ 49  55  54]
  ...
  [ 52  61  58]
  [ 47  56  53]
  [ 46  55  52]]]


True

In [ ]:
(2092, 1664, 727, 1224)